Import packages

In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib.pyplot import imshow
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers

In [2]:
def get_session(gpu_fraction=0.25):    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction, allow_growth=True)    
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.set_session(get_session())


Declare variables to be used later.  MAKE SURE TO DOUBLE CHECK THESE

In [2]:
# Assume all images are square
image_size = 230

# Need to change this to match category count
num_categories = 8


# Change this for number of images to be in train and test set
trainsize = 2000
testsize = 10

Import image data and convert response to an integer

In [4]:
genrePath = os.path.join('/team', 'scratch', 'album-ai', 'data', 'genres.csv')

genres = pd.read_csv(genrePath)
genres = genres.loc[genres['genre'] != 'unknown']

genre_dict = {'pop':0, 
              'rock':1, 
              'country':2,
              'folk':2,
              'rap':3,
              'hip hop':3,
              'metal':4, 
              'indie':5, 
              'electronic':6, 
              'classical':7}

genres['genre_id'] = genres['genre'].map(genre_dict)

Create function to convert image to numpy array

In [5]:
def jpg_image_to_array(image_path):
    """
    Loads JPEG image into 3D Numpy array of shape 
    (width, height, channels)
    """
    with Image.open(image_path) as image:         
        im_arr = np.frombuffer(image.tobytes(), dtype=np.uint8)
        im_arr = im_arr.reshape((image.size[1], image.size[0], 3))                                   
    return im_arr

Pull actual resized image data.  Array will be 230 by 230 by 3

In [6]:
def loaddata():
    covers = os.path.join('/team', 'scratch', 'album-ai', 'data', 'resizedArt')

    trainx = np.array([], dtype=np.uint8)
    trainy = []
    
    loggingCount = 0

    for file in list(genres['id'])[0:trainsize]:
        image_arr = jpg_image_to_array(os.path.join(covers, str(file) + '.jpg'))
        image_arr = image_arr[np.newaxis, :]
        trainx = np.vstack((trainx, image_arr)).astype('uint8') if trainx.size else image_arr
        trainy.append(int(genres[genres['id'] == file]['genre_id']))
        
        loggingCount += 1
        if loggingCount % 100 == 0:
            print('{} out of {} training albums imported'.format(loggingCount, trainsize))

    trainy = np.asarray(trainy)

    testx = np.array([], dtype=np.uint8)
    testy = []
    
    loggingCount = 0

    for file in list(genres['id'])[trainsize:trainsize + testsize]:
        image_arr = jpg_image_to_array(os.path.join(covers, str(file) + '.jpg'))
        image_arr = image_arr[np.newaxis, :]
        testx = np.vstack((testx, image_arr)).astype('uint8') if testx.size else image_arr
        testy.append(int(genres[genres['id'] == file]['genre_id']))
        
        loggingCount += 1
        if loggingCount % 100 == 0:
            print('{} out of {} testing albums imported'.format(loggingCount, testsize))

    testy = np.asarray(testy)
    
    return trainx, trainy, testx, testy

Show test image

In [7]:
# im = Image.fromarray(testx[1])

%matplotlib inline
# imshow(im)

In [8]:
def transformdata():
    trainx, trainy, testx, testy = loaddata()
    trainx = trainx/255.
    testx = testx/255.
    trainy = to_categorical(trainy,num_categories)
    testy = to_categorical(testy,num_categories)
    trainx = np.reshape(trainx,(len(trainx),image_size*image_size*3))
    testx = np.reshape(testx,(len(testx),image_size*image_size*3))
    return trainx,trainy,testx,testy

In [9]:
def build_model1():
    inp = Input(shape=(image_size*image_size*3,))
    d1 = Dense(128,activation='relu')(inp)
    d2 = Dense(128,activation='relu')(d1)
    out = Dense(num_categories,activation='softmax')(d2)
    model = Model(inputs=inp,outputs=out)
    sgd = optimizers.SGD(lr=0.01)
    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [10]:
def main(trainx, trainy, testx, testy):
    address = './models/'
    if not os.path.exists(address):
        os.makedirs(address)
    saved_model = address + "model.h5"
    model = build_model1()
    stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    save = ModelCheckpoint(saved_model, monitor='val_loss', verbose=0, save_best_only=True)
    print("Start training")
    hist = model.fit(trainx,trainy,
                     callbacks=[stop,save],validation_split=0.2,
                     epochs=50,batch_size=100)
    print('Delete current model')
    del model
    print('Load saved model')
    model = load_model(saved_model)
    score = model.evaluate(testx,testy)
    print('Test loss: {:08.5f} ...Test accuracy: {:06.2f}%'.format(score[0],score[1]*100)) 
    return

This cell imports the training and testing data

In [11]:
# Only need to run this cell when changing the training and testing data
# Takes several minutes to run for large numbers of images

trainx, trainy, testx, testy = transformdata()

100 out of 2000 training albums imported
200 out of 2000 training albums imported
300 out of 2000 training albums imported
400 out of 2000 training albums imported
500 out of 2000 training albums imported
600 out of 2000 training albums imported
700 out of 2000 training albums imported
800 out of 2000 training albums imported
900 out of 2000 training albums imported
1000 out of 2000 training albums imported
1100 out of 2000 training albums imported
1200 out of 2000 training albums imported
1300 out of 2000 training albums imported
1400 out of 2000 training albums imported
1500 out of 2000 training albums imported
1600 out of 2000 training albums imported
1700 out of 2000 training albums imported
1800 out of 2000 training albums imported
1900 out of 2000 training albums imported
2000 out of 2000 training albums imported


This creates the model and trains it

In [12]:
# Run this cell when changing the model

main(trainx, trainy, testx, testy)

Instructions for updating:
Colocations handled automatically by placer.
Start training
Train on 1600 samples, validate on 400 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1600/1600 [==============================] - 4s 3ms/sample - loss: 5.0538 - acc: 0.1931 - val_loss: 1.9815 - val_acc: 0.0325
Epoch 2/50
1600/1600 [==============================] - 2s 1ms/sample - loss: 1.9478 - acc: 0.2544 - val_loss: 2.0713 - val_acc: 0.3750
Epoch 3/50
1600/1600 [==============================] - 2s 1ms/sample - loss: 1.9262 - acc: 0.2706 - val_loss: 2.1187 - val_acc: 0.0350
Epoch 4/50
1600/1600 [==============================] - 2s 1ms/sample - loss: 1.9240 - acc: 0.2750 - val_loss: 2.1348 - val_acc: 0.3775
Epoch 5/50
1600/1600 [==============================] - 2s 1ms/sample - loss: 1.9210 - acc: 0.2575 - val_loss: 2.1982 - val_acc: 0.3750
Epoch 6/50
1600/1600 [==============================] - 2s 1ms/sample - loss: 1.9104 - acc: 0.2694 - val_loss: 2.0543 - val_acc: 0.3900
Ep